In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.signal import savgol_filter
from lmfit.models import LorentzianModel, ConstantModel
from astropy.io import fits
from pathlib import Path

# Load the Swift flux data
swiftflux = "SWIFTJ1727.8-1613.lc.fits"
pathname = Path("~/Downloads").expanduser()
swiftfilename = list(pathname.rglob(swiftflux))

if not swiftfilename:
    raise FileNotFoundError(f"File {swiftflux} not found in {pathname}")

sdata, sheader = fits.getdata(swiftfilename[0], header=True)
swift_times = sdata['TIME']
swift_flux = sdata['RATE']

# Initialize lists to store statistics and time
center_frequencies = []
widths = []
intensities = []
cerr = []
werr = []
#ierr = []


for i, datasegment in enumerate(lcsegments):
    # Access the specific lightcurve segment
    #datasegment = lcsegments[n]
    n = prev_fast_FFT_len(len(datasegment))

    # Trim to the last n values
    datasegment = datasegment[-n:]
    duration = datasegment['time'].ptp()
    filtered_data = datasegment['norm']
    ntimes = len(filtered_data)

    # Do the Fourier transform, and get the corresponding frequencies and powers
    fnorm = sp.fft.rfft(filtered_data, norm='forward')
    freqs = abs(sp.fft.rfftfreq(ntimes, timebin))
    fpower = np.abs(fnorm)**2

    qpo_index = np.argmax(fpower)
    qpo_frequency = freqs[qpo_index]
    qpo_intensity = fpower[qpo_index]

    # Prepare data for fitting
    x = freqs
    y = fpower
        
    w = savgol_filter(y, 51, 2)

    # Define multiple Lorentzian models
    mod1 = LorentzianModel(prefix='l1_')
    mod2 = LorentzianModel(prefix='l2_')
    mod3 = ConstantModel(prefix='c1_')
    mod = mod1 + mod2 + mod3

    # Initial guesses
    pars = mod1.guess(w, x=x)
    pars.update(mod2.make_params())
    pars.update(mod3.make_params())

    # Fit the model
    out = mod.fit(w, pars, x=x)
    print(out.fit_report(min_correl=1))

    # Extract statistics
    center_frequencies.append(out.params['l1_center'].value)
    widths.append(out.params['l1_fwhm'].value)
    intensities.append(qpo_intensity)
    cerr.append(out.params['l1_center'].stderr)
    werr.append(out.params['l1_fwhm'].stderr)
    #ierr.append(out.params['l1_sigma'].stderr)

    # Plot the data and the fit for this segment
    #plt.figure(figsize=(10, 6))
    # plt.plot(x, y, label='Data')
    #plt.plot(x, out.best_fit, label='Fit')
    #plt.xlabel('Frequency (Hz)')
    #plt.ylabel('Power')
    # plt.title(f'Segment {n}')
    # plt.legend()
    # plt.show()

# Replace None values in error arrays with zeros
cerr = [0 if v is None else v for v in cerr]
werr = [0 if v is None else v for v in werr]
#ierr = [0 if v is None else v for v in ierr]

# Convert lists to numpy arrays
center_frequencies = np.array(center_frequencies)
widths = np.array(widths)
intensities = np.array(intensities)
cerr = np.array(cerr)
werr = np.array(werr)
#ierr = np.array(ierr)

# sorted_indices = np.argsort(swift_times)
# center_frequencies = center_frequencies[sorted_indices]
# widths = widths[sorted_indices]
# intensities = intensities[sorted_indices]
    

# Ensure all arrays have the same length
min_len = min(len(swift_times), len(center_frequencies), len(widths), len(intensities))

swift_times = swift_times[:min_len]
center_frequencies = center_frequencies[:min_len]
widths = widths[:min_len]
intensities = intensities[:min_len]
cerr = cerr[:min_len]
werr = werr[:min_len]
#ierr = ierr[:min_len]

In [ ]:
QPO_segments = [36,42, 49, 64, 72, 73, 78, 79, 85, 86, 90, 117, 122, 128, 134, 141, 142, 157, 162, 173, 168, 181, 183, 203, 210, 216, 233, 240, 257, 272, 281, 294 ,307, 323]

In [ ]:
if False:
    import numpy as np
    import matplotlib.pyplot as plt
    import scipy as sp
    from scipy.signal import savgol_filter
    from lmfit.models import LorentzianModel
    from astropy.io import fits
    from pathlib import Path
    import swiftbat
    
    # Load the Swift flux data
    swiftflux = "SWIFTJ1727.8-1613.lc.fits" # time
    pathname = Path("~/Downloads").expanduser()
    swiftfilename = list(pathname.rglob(swiftflux))
    
    # Debug: Print the paths found
    print("Swift filename paths found:", swiftfilename)
    
    if not swiftfilename:
        raise FileNotFoundError(f"File {swiftflux} not found in {pathname}")
    
    sdata, sheader = fits.getdata(swiftfilename[0], header=True)
    swift_times = sdata['TIME']
    swift_flux = sdata['RATE']
    # Initialize lists to store statistics and time
    times = []
    peak_frequencies = []
    widths = []
    amplitudes = []
    
    plt.close('all')
    for n in QPO_segments:
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))  # Adjust dimensions as needed
        
        # Access the specific lightcurve segment
        datasegment = lcsegments[n]
        n = prev_fast_FFT_len(len(datasegment))
        
        # Trim to the last n values
        datasegment = datasegment[-n:]
        duration = datasegment['time'].ptp()
        filtered_data = datasegment['norm']
        ntimes = len(filtered_data)
        
        # Do the Fourier transform, and get the corresponding frequencies and powers
        fnorm = sp.fft.rfft(filtered_data, norm='forward')
        freqs = abs(sp.fft.rfftfreq(ntimes, timebin))
        fpower = np.abs(fnorm)**2
        
        # Plot the results
        nice_plot(freqs, fpower, axes, label=f'{n}')
        axes.legend()
        
        # Prepare data for fitting
        x = freqs
        y = fpower
        w = savgol_filter(y, 101, 2)
        
        # Define multiple Lorentzian models
        mod1 = LorentzianModel(prefix='l1_')
        mod2 = LorentzianModel(prefix='l2_')
        mod3 = ConstantModel(bkg='c1_')
        mod = mod1 + mod2 + mod3
        
        # Initial guesses
        pars = mod1.guess(w, x=x)
        pars.update(mod2.make_params())
        pars.update(mod3.make_params())
        
        # Fit the model
        out = mod.fit(w, pars, x=x)
        print(out.fit_report(min_correl=1))
        
        # Extract statistics
        peak_frequencies.append(out.params['l1_center'].value)
        widths.append(out.params['l1_sigma'].value)
        amplitudes.append(out.params['l1_amplitude'].value)
        
        # Store the corresponding time (using the midpoint of the segment)
        times.append(datasegment['time'].mean())
        
        # Plot the results
        plt.plot(x, w, label='Filtered Data')
        plt.plot(x, out.init_fit, 'k--', label='Initial Fit')
        plt.plot(x, out.best_fit, 'r-', label='Best Fit')
    
    plt.show()
    
    # Convert times to numpy array for sorting and plotting
    times = np.array(times)
    peak_frequencies = np.array(peak_frequencies)
    widths = np.array(widths)
    amplitudes = np.array(amplitudes)
    
    # Sort by time
    sorted_indices = np.argsort(times)
    times = times[sorted_indices]
    peak_frequencies = peak_frequencies[sorted_indices]
    widths = widths[sorted_indices]
    amplitudes = amplitudes[sorted_indices]
    
    # Plot statistics vs time
    fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 12), sharex=True)
    
    axes[0].plot(times, peak_frequencies, label='Peak Frequency', color='blue')
    axes[0].set_ylabel('Peak Frequency (Hz)')
    axes[0].legend()
    
    axes[1].plot(times, widths, label='Width', color='green')
    axes[1].set_ylabel('Width')
    axes[1].legend()
    
    axes[2].plot(times, amplitudes, label='Amplitude', color='red')
    axes[2].set_ylabel('Amplitude')
    axes[2].legend()
    
    axes[3].plot(swift_times, swift_flux, label='Swift Flux', color='purple')
    axes[3].set_xlabel('Time (MJD)')
    axes[3].set_ylabel('Swift Flux (counts/s/cm^2)')
    axes[3].legend()
    
    plt.tight_layout()
    plt.show()


In [ ]:
if False:
    import numpy as np
    import matplotlib.pyplot as plt
    import scipy as sp
    from scipy.signal import savgol_filter
    from lmfit.models import LorentzianModel
    
    # Initialize lists to store statistics and time
    times = []
    peak_frequencies = []
    widths = []
    amplitudes = []
    
    plt.close('all')
    for n in QPO_segments:
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))  # Adjust dimensions as needed
        
        # Access the specific lightcurve segment
        datasegment = lcsegments[n]
        n = prev_fast_FFT_len(len(datasegment))
        
        # Trim to the last n values
        datasegment = datasegment[-n:]
        duration = datasegment['time'].ptp()
        filtered_data = datasegment['norm']
        ntimes = len(filtered_data)
        
        # Do the Fourier transform, and get the corresponding frequencies and powers
        fnorm = sp.fft.rfft(filtered_data, norm='forward')
        freqs = abs(sp.fft.rfftfreq(ntimes, timebin))
        fpower = np.abs(fnorm)**2
        
        # Plot the results
        nice_plot(freqs, fpower, axes, label=f'{n}')
        axes.legend()
        
        # Prepare data for fitting
        x = freqs
        y = fpower
        w = savgol_filter(y, 101, 2)
        
        # Define multiple Lorentzian models
        mod1 = LorentzianModel(prefix='l1_')
        mod2 = LorentzianModel(prefix='l2_')
        mod = mod1 + mod2 
        
        # Initial guesses
        pars = mod1.guess(w, x=x)
        pars.update(mod2.make_params())
        
        # Fit the model
        out = mod.fit(w, pars, x=x)
        print(out.fit_report(min_correl=1))
        
        # Extract statistics
        peak_frequencies.append(out.params['l1_center'].value)
        widths.append(out.params['l1_sigma'].value)
        amplitudes.append(out.params['l1_amplitude'].value)
        
        # Store the corresponding time (using the midpoint of the segment)
        times.append(datasegment['time'].mean())
        
        # Plot the results
        plt.plot(x, w, label='Filtered Data')
        plt.plot(x, out.init_fit, 'k--', label='Initial Fit')
        plt.plot(x, out.best_fit, 'r-', label='Best Fit')
    
    plt.show()
    
    # Convert times to numpy array for sorting and plotting
    times = np.array(times)
    peak_frequencies = np.array(peak_frequencies)
    widths = np.array(widths)
    amplitudes = np.array(amplitudes)
    
    # Sort by time
    sorted_indices = np.argsort(times)
    times = times[sorted_indices]
    peak_frequencies = peak_frequencies[sorted_indices]
    widths = widths[sorted_indices]
    amplitudes = amplitudes[sorted_indices]
    
    # Plot statistics vs time
    plt.figure(figsize=(12, 8))
    plt.subplot(3, 1, 1)
    plt.plot(times, peak_frequencies, label='Peak Frequency')
    plt.ylabel('Peak Frequency (Hz)')
    plt.legend()
    
    plt.subplot(3, 1, 2)
    plt.plot(times, widths, label='Width')
    plt.ylabel('Width')
    plt.legend()
    
    plt.subplot(3, 1, 3)
    plt.plot(times, amplitudes, label='Amplitude')
    plt.ylabel('Amplitude')
    plt.xlabel('Time')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Plot source flux vs time on the same scale
    # Assuming `source_flux` is a 1D array of flux values and `source_times` is the corresponding times
    plt.figure(figsize=(12, 8))
    plt.plot(source_times, source_flux, label='Source Flux', color='purple')
    plt.xlabel('Time')
    plt.ylabel('Source Flux')
    plt.legend()
    plt.show()


In [ ]:
 if False:   
    import numpy as np
    import matplotlib.pyplot as plt
    import scipy as sp
    
    # Initialize variables
    num_segments = len(lcsegments)
    np.seterr(divide='ignore', invalid='ignore')
    
    # Prepare to accumulate power spectra
    # Assuming the longest segment defines the length of the frequency array
    max_ntimes = max(len(segment['norm']) for segment in lcsegments)
    max_freqs = sp.fft.rfftfreq(prev_fast_FFT_len(max_ntimes), timebin)
    power_matrix = np.zeros((num_segments, len(max_freqs)))
    
    # Loop through segments
    for i, datasegment in enumerate(lcsegments):
        n = prev_fast_FFT_len(len(datasegment))
        datasegment = datasegment[-n:]
        filtered_data = datasegment['norm']
        ntimes = len(filtered_data)
        fnorm = sp.fft.rfft(filtered_data, norm='forward')
        fpower = np.abs(fnorm)**2
        
        # Calculate freqs for the current segment
        freqs = sp.fft.rfftfreq(ntimes, timebin)
        
        # Define a high-frequency range for noise estimation based on current freqs
        high_freq_range = freqs > freqs[-1] * 0.8
        
        # Estimate noise level in the high-frequency range
        noise_level = np.std(fpower[high_freq_range])
        
        # Normalize the power by the noise level
        if noise_level > 0:
            fpower /= noise_level
        
        # Ensure fpower has the same length as max_freqs and store in power_matrix
        if len(fpower) != len(max_freqs):
            fpower = np.interp(max_freqs, freqs, fpower)
        
        # Apply log scaling to maintain variations
        power_matrix[i, :] = np.log1p(fpower)
    
    # Plot each segment's power spectrum as a row in the image
    plt.figure(figsize=(12, 8))
    for i in range(num_segments):
        plt.plot(max_freqs, power_matrix[i, :], label=f'Segment {i}')
    plt.yscale('log')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Brightness')
    plt.title('Power Spectra of Segments')
    plt.legend()
    plt.show()
    
    # Plot the accumulated power matrix with imshow
    plt.figure(figsize=(12, 8))
    plt.imshow(power_matrix, aspect='auto', origin='lower', extent=[max_freqs[0], max_freqs[-1], 0, num_segments], cmap='viridis')
    plt.colorbar(label='Brightness')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Segment')
    plt.title('2D Power Spectrum with Segment Index and Frequency')
    plt.show()


In [ ]:
if False:
    # Load the data
    swiftflux = "SWIFTJ1727.8-1613.lc.fits"  # time
    pathname = Path("~/Downloads").expanduser()
    
    # Check if the path is correct
    print(f"Looking for files in: {pathname}")
    
    swiftfilename = list(pathname.rglob(swiftflux))
    
    # Check if any files were found
    if not swiftfilename:
        print(f"No files found matching: {swiftflux} in {pathname}")
    else:
        print(f"Files found: {swiftfilename}")
    
    # Assuming only one file is found
    sdata, sheader = fits.getdata(swiftfilename[0], header=True)
    
    # tzero = swiftbat.met2mjd(swiftbat.string2met('2023-08-27T00:00:00'))
    # print(tzero)
    
    sdataflux = sdata['RATE']
    sdatatime = sdata['TIME']
    
    # If datasegment is not defined, assume the entire data segment
    datasegment = sdata  # Using entire data as segment
    timebin = 1  # Define the time bin, for example purposes set to 1
    
    time_values = datasegment['TIME']
    rate_values = datasegment['RATE']  
    
    # Preprocess the data
    # Detrending 
    # flux_detrended = sdataflux - np.mean(sdataflux)
    
    # Identify QPO peaks using Welch's method for PSD
    fs = 1 / (time_values[1] - time_values[0])  # Sampling frequency
    frequencies, power_spectrum = welch(rate_values, fs=fs, nperseg=min(256, len(rate_values)))
    
    # Find peaks in the power spectrum
    peaks, _ = find_peaks(power_spectrum, height=0)
    
    # Extract QPO parameters
    qpo_frequencies = frequencies[peaks]
    qpo_strengths = power_spectrum[peaks]
    qpo_widths = []  # Placeholder for peak widths calculation
    
    # Calculate widths (FWHM) - simple approximation
    for peak in peaks:
        half_max = power_spectrum[peak] / 2
        left_idx = np.where(power_spectrum[:peak] <= half_max)[0]
        right_idx = np.where(power_spectrum[peak:] <= half_max)[0]
        
        if len(left_idx) > 0 and len(right_idx) > 0:
            width = frequencies[right_idx[0] + peak] - frequencies[left_idx[-1]]
            qpo_widths.append(width)
        else:
            qpo_widths.append(np.nan)
    
    # Plot QPO parameters against time
    plt.figure(figsize=(10, 6))
    
    plt.subplot(4, 1, 1)
    plt.plot(sdatatime, sdataflux, color = 'r', label='Source Flux')
    plt.xlabel('Time')
    plt.ylabel('Flux')
    plt.legend()
    
    plt.subplot(4, 1, 2)
    plt.plot(time_values[:len(qpo_strengths)], qpo_strengths, color = 'g', label='QPO Strength')
    plt.xlabel('Time')
    plt.ylabel('Strength')
    plt.legend()
    
    plt.subplot(4, 1, 3)
    plt.plot(time_values[:len(qpo_widths)], qpo_widths, label='QPO Width')
    plt.xlabel('Time')
    plt.ylabel('Width')
    plt.legend()
    
    plt.subplot(4, 1, 4)
    plt.plot(time_values[:len(qpo_frequencies)], qpo_frequencies, color = 'purple', label='QPO Frequency')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.legend()
    
    plt.tight_layout()
    plt.show()


In [ ]:
if False:
    # Define tzero (assuming swiftbat.string2met is defined elsewhere)
    tzero = swiftbat.string2met('2023-08-27T05:22:01')
    
    # Assuming datasegment is your data containing 'TIME' and 'COUNTS' fields
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 6), sharex=True)
    
    # Number of segments to split the data into
    segpieces = 4
    
    # List to store FFT results
    stacked_freq_values = []
    stacked_counts_fft = []
    
    # Plotting frequency spectrum for each segment
    for i, istart in enumerate(np.arange(0, len(datasegment), len(datasegment) // segpieces)):
        sl = slice(istart, istart + len(datasegment) // segpieces)
        time_values = datasegment['time'][sl]
        counts_data = datasegment['COUNTS'][sl]
    
        # Perform FFT on COUNTS data
        counts_fft = scipy.fft.rfft(counts_data)
        freq_values = scipy.fft.rfftfreq(len(counts_data), d=(time_values[-1] - time_values[0]) / len(counts_data))
    
        # Rescale to the same frequency range
        max_freq = np.max(freq_values)
        freq_interp = np.linspace(0, max_freq, len(freq_values))
    
        # Ensure counts_fft is 1D
        counts_fft_abs = np.abs(counts_fft)
        if counts_fft_abs.ndim > 1:
            counts_fft_abs = counts_fft_abs.flatten()
    
        # Print shapes for debugging
        print(f"freq_interp shape: {freq_interp.shape}")
        print(f"freq_values shape: {freq_values.shape}")
        print(f"counts_fft_abs shape: {counts_fft_abs.shape}")
    
        # Ensure lengths match before interpolation
        min_len = min(len(freq_values), len(counts_fft_abs))
        freq_values = freq_values[:min_len]
        counts_fft_abs = counts_fft_abs[:min_len]
    
        # Interpolating and stacking FFT results
        counts_fft_rescaled = np.interp(freq_interp, freq_values, counts_fft_abs)
    
        # Plot the frequency spectrum for the current segment
        axes[0].plot(freq_values, counts_fft_abs, label=f"Segment {i}")
    
        # Stack FFT results
        stacked_freq_values.append(freq_interp)  # Use freq_interp to ensure consistent frequency range
        stacked_counts_fft.append(counts_fft_rescaled)
        break
    
    axes[0].legend()
    axes[0].set_title("Frequency Spectrum for Segments")
    
    # Stacking FFT results
    stacked_freq_values = np.concatenate(stacked_freq_values)
    stacked_counts_fft = np.array(stacked_counts_fft)
    
    # Plot stacked and rescaled frequency spectrum
    axes[1].imshow(stacked_counts_fft, aspect='auto', origin='lower', extent=[np.min(stacked_freq_values), np.max(stacked_freq_values),0, segpieces])
    axes[1].set_title("Stacked and Rescaled Frequency Spectrum")
    axes[1].set_xlabel("Frequency (Hz)")
    axes[1].set_ylabel("Segment")
    axes[1].invert_yaxis()
    
    plt.show()


In [ ]:
if False:
    tzero = swiftbat.string2met('2023-08-27T00:00:00')
    fapprox = .027139
    cycle, phase = np.divmod((datasegment['time'] - tzero) * fapprox, 1)
    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True)
    
    print(filtered_data.shape)
    print(datasegment['time'].shape)
    
    # plotting just the 1300s segment
    #rate = np.sum(datasegment['COUNTS'][0:2]) / timebin
    #print(len(datasegment))
    
    segpieces = 4
    # Break the segment into 4 pieces
    pointsperplot = 2 * int(1 / (0.064 * fapprox))
    
    print(f"Length of datasegment: {len(datasegment)}")
    
    # For each segment, plot 2 cycles of data
    for istart in np.arange(0, len(datasegment), 1 + len(datasegment) // segpieces):
        sl = slice(istart, istart + pointsperplot)
        time_values = datasegment['time']
    if isinstance(time_values, np.ndarray):
        phase_values = phase[sl]
        rate_values = filtered_data[sl] / timebin
        axes[0].plot(phase_values, rate_values, ".", label=f"{time_values[0] - tzero:.0f}") 
    else:
        print("Warning: time_values is not an array")   
    axes[0].set_title(f"For Longest Segment: Change in Rate Over a Phase with Freq of {fapprox} Hz")
    
    # 1 cycle of data for all 11 data segments
    for datasegment in np.split(obsdata, splitlocs):
        n = prev_fast_FFT_len(len(datasegment) - int(60 / timebin))
        datasegment = datasegment[-n:]
        duration = datasegment['time'].ptp()
        print("time: ", datasegment['time'][0])
        print(f"{duration:.3f} seconds after trimming")
        segrate = filtered_data / timebin
        segcycle, segphase = np.divmod((datasegment['time'] - tzero) * fapprox, 1)
        print(len(datasegment))
        axes[1].plot(segphase[:3500], segrate[:3500], ".", label="Segmentation")
    
    axes[1].legend()
    axes[1].set_title(f"For All segments: Change in Rate Over a Phase with Freq of {fapprox} Hz")
    
    fapprox = .641961
    
    for datasegment in np.split(obsdata, splitlocs):
        n = prev_fast_FFT_len(len(datasegment) - int(60 / timebin))
        datasegment = datasegment[-n:]
        duration = datasegment['time'].ptp()
        print("time: ", datasegment['time'][0])
        print(f"{duration:.3f} seconds after trimming")
        segrate = filtered_data/ timebin
        segcycle, segphase = np.divmod((datasegment['time'] - tzero) * fapprox, 1)
        print(len(datasegment))
        axes[2].plot(segphase[:3500], segrate[:3500], ".", label="Segmentation")
        
    axes[2].legend()
    axes[2].set_title(f"For All segments: Change in Rate Over a Phase with Freq of {fapprox} Hz")
    plt.show()

In [ ]:
if False:
    plt.close('all')
    fig, ax = plt.subplots(figsize=(10, 6))
    
    clip_level = 1e-3
    # Iterate through segments and plot each power spectrum on the same axes
    for i, datasegment in enumerate(lcsegments):
        n = prev_fast_FFT_len(len(datasegment))
        datasegment = datasegment[-n:]
        filtered_data = datasegment['norm']
        ntimes = len(filtered_data)
        fnorm = sp.fft.rfft(filtered_data, norm='forward')
        fpower = np.abs(fnorm)**2
        freqs = sp.fft.rfftfreq(ntimes, timebin)
        freqs[freqs < clip_level] = np.nan
        # Call nice_plot function to plot the current segment
        nice_plot(freqs, fpower, ax)
        print(swiftbat.met2datetime(datasegment['time'][0]))
    
    # Set log scale, limits, labels, title, legend, and grid
    #ax.set_yscale('log')
    #ax.set_ylim(bottom=0, top=None)  
    #ax.set_xlim(left=0.05, right=2)  
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Power') 
    ax.set_title('Overlay of Power Spectra from Segments')
    ax.grid(True)
    
    # Show the plot
    plt.show()

In [ ]:
if False:
    lc = Lightcurve()
    
    
    plt.close('all')
    dys = stingray.DynamicalPowerspectrum(lc=lc, segment_size = 10, norm="leahy")
    print("Dynamical Power Spectrum Statistics:")
    print(f"Min value: {np.min(dys.dyn_ps)}")
    print(f"Max value: {np.max(dys.dyn_ps)}")
    print(f"Mean value: {np.mean(dys.dyn_ps)}")
    
    # Plot the initial dynamical power spectrum
    plt.figure()
    plt.imshow(dys.dyn_ps, aspect="auto", vmin=0, vmax=100, origin="lower",
               extent=[min(dys.time), max(dys.time), min(dys.freq), max(dys.freq)])
    plt.colorbar()
    plt.title('Initial Dynamical Power Spectrum')
    plt.show()
    
    # Rebin the frequency axis
    dys.rebin_frequency(df_new=5, method="mean")


    # Plot the re-binned dynamical power spectrum
    plt.figure()
    plt.imshow(dys.dyn_ps, aspect="auto", vmin=20, vmax=80, origin="lower", 
               extent=[min(dys.time), max(dys.time), min(dys.freq), max(dys.freq)])
    plt.colorbar()
    plt.title('Re-binned Dynamical Power Spectrum')
    plt.ylim(1, 7)
    plt.show()
    
    dys.rebin_time(dt_new=dys.dt*3, method="mean")
    plt.figure()
    plt.imshow(dys.dyn_ps, aspect="auto", vmin=20, vmax=80, origin="lower",
              extent=[min(dys.time), max(dys.time), min(dys.freq), max(dys.freq)])
    plt.colorbar()
    plt.ylim(1,2)
    plt.show()
    
    pos = dys.trace_maximum(min_freq=0, max_freq=2)
    plt.figure()
    plt.imshow(dys.dyn_ps, aspect="auto", vmin=20, vmax=80, origin="lower",
              extent=[min(dys.time), max(dys.time), min(dys.freq), max(dys.freq)],
              alpha=0.9)
    plt.colorbar()
    plt.plot(dys.time, dys.freq[pos], 'r', lw=2, alpha=1)
    plt.ylim(1,2)
    
    #plt.imshow(dys.dyn_ps, aspect="auto", vmin=1, vmax=3, origin="lower",
              #extent=[min(dys.time), max(dys.time), min(dys.freq), max(dys.freq)])
    #plt.colorbar()
    #dys.rebin_frequency(df_new=5, method="mean")
    #plt.imshow(dys.dyn_ps, aspect="auto", vmin=1.8, vmax=2.8, origin="lower",
               #extent=[min(dys.time), max(dys.time), min(dys.freq), max(dys.freq)])
    #plt.colorbar()
    #plt.ylim(.0001,.001)
    #plt.show()


In [ ]:
if False:
    from stingray.modeling import PSDPosterior
    import scipy.stats
    from stingray.modeling import set_logprior
    lpost = PSDPosterior(ps.freq, ps.power, plc, m=ps.m)
    
    # flat prior for the power law index
    p_alpha = lambda alpha: ((-1. <= alpha) & (alpha <= 5.))
    
    # flat prior for the power law amplitude
    p_amplitude = lambda amplitude: ((0.01 <= amplitude) & (amplitude <= 10.0))
    
    # normal prior for the white noise parameter
    p_whitenoise = lambda white_noise: scipy.stats.norm(2.0, 0.1).pdf(white_noise)
    
    priors = {}
    priors["alpha_0"] = p_alpha
    priors["amplitude_0"] = p_amplitude
    priors["amplitude_1"] = p_whitenoise
    lpost.logprior = set_logprior(lpost, priors)
    lpost = PSDPosterior(ps.freq, ps.power, plc, priors=priors, m=ps.m)
    parest = PSDParEst(ps, fitmethod='BFGS', max_post=True)
    res = parest.fit(lpost, starting_pars)
    print("best-fit parameters:")
    for p,e in zip(res.p_opt, res.err):
        print("%.4f +/- %.4f"%(p,e))
    
    res.print_summary(lpost)
    sample = parest.sample(lpost, res.p_opt, cov=res.cov, nwalkers=400,
                 niter=100, burnin=300, namestr="psd_modeling_test")
    fig = sample.plot_results(nsamples=1000, fig=None, save_plot=True,
                        filename="modeling_tutorial_mcmc_corner.pdf")

In [ ]:


# Load the Swift flux data
swiftflux = "SWIFTJ1727.8-1613.lc.fits"
pathname = Path("~/Downloads").expanduser()
swiftfilename = list(pathname.rglob(swiftflux))

if not swiftfilename:
    raise FileNotFoundError(f"File {swiftflux} not found in {pathname}")

sdata, sheader = fits.getdata(swiftfilename[0], header=True)
swift_times = sdata['TIME']
swift_flux = sdata['RATE']

# Initialize lists to store statistics and time
times = []
center_frequencies = []
widths = []
intensities = []
cerr = []
werr = []
#ierr = []

for i, datasegment in enumerate(lcsegments[100:110]):
    out, freqs, fpower = fit_frequencies(datasegment)
    print(out.fit_report(min_correl=1))
    plot_fit(out, freqs, fpower, i)
    if out.params['l2_center'] < .2:
        continue
        

    # Extract statistics
    # center_frequencies.append(out.params['l1_center'].value)
    # widths.append(out.params['l1_fwhm'].value)
    # intensities.append(qpo_intensity)
    # cerr.append(out.params['l1_center'].stderr)
    # werr.append(out.params['l1_fwhm'].stderr)

    
    #ierr.append(out.params['l1_sigma'].stderr)

    # Plot the data and the fit for this segment
    #plt.figure(figsize=(10, 6))
    # plt.plot(x, y, label='Data')
    #plt.plot(x, out.best_fit, label='Fit')
    #plt.xlabel('Frequency (Hz)')
    #plt.ylabel('Power')
    # plt.title(f'Segment {n}')
    # plt.legend()
    # plt.show()

# Replace None values in error arrays with zeros
# cerr = [0 if v is None else v for v in cerr]
# werr = [0 if v is None else v for v in werr]
# #ierr = [0 if v is None else v for v in ierr]

# # Convert lists to numpy arrays
# center_frequencies = np.array(center_frequencies)
# widths = np.array(widths)
# intensities = np.array(intensities)
# cerr = np.array(cerr)
# werr = np.array(werr)
# #ierr = np.array(ierr)

# # sorted_indices = np.argsort(swift_times)
# # center_frequencies = center_frequencies[sorted_indices]
# # widths = widths[sorted_indices]
# # intensities = intensities[sorted_indices]
    

# # Ensure all arrays have the same length
# min_len = min(len(swift_times), len(center_frequencies), len(widths), len(intensities))

# swift_times = swift_times[:min_len]
# center_frequencies = center_frequencies[:min_len]
# widths = widths[:min_len]
# intensities = intensities[:min_len]
# cerr = cerr[:min_len]
# werr = werr[:min_len]
#ierr = ierr[:min_len]